# Demonstate Functionality of the *parse_input* module that is used by the *mpc_nbody* module
 - Primarily written by M. Alexandersen, with some additions from M. Payne 

In [4]:
import sys
import os
from filecmp import cmp
import numpy as np
import pytest
import importlib
from astroquery.jplhorizons import Horizons

# parent directory is */cheby_checker
HEAD_DIR = os.path.dirname(os.path.realpath(os.getcwd())) 
sys.path.append(os.path.join(HEAD_DIR))
print(f' HEAD_DIR: {HEAD_DIR} ')

# import nbody-related code from main cheby_checker directory
from cheby_checker import  parse_input
importlib.reload(parse_input)

 HEAD_DIR: /Users/matthewjohnpayne/Envs/cheby_checker 


<module 'cheby_checker.parse_input' from '/Users/matthewjohnpayne/Envs/cheby_checker/cheby_checker/parse_input.py'>

In [5]:
DATA_DIR = os.path.join(os.path.dirname(os.getcwd()), 'dev_data')
test_files = [os.path.join(DATA_DIR, file)
              for file in ['30101.eq0_horizons', '30102.eq0_horizons']]
au_km = 149597870.700  # This is now a definition

In [7]:
# For more information, uncomment next line:
# help(parse_input)

# Explicit Test Code Exists in mpc_nbody/tests

 - test_parse_input.py
 - test_everything.py
 
 Run these using pytest ...
  - cd ../tests
  - pytest test_parse_input.py


# Top-level: "parse_input.ParseElements" - Parse, convert & store elements.
 - This takes input as ele220 (not yet implemented) or OrbFit files (".fel" or ".eq" files, which I actually think have the same format), parses the elements and time, converts that to TBD and Barycentric Equatorial elements, and saves that to a holman_ic file for input into the actual nbody integrator. 
 - This calls a number of under-lying functionalities

In [8]:
%time
# First, let's do everything at once by initiating with input:
_ = parse_input.ParseElements(input_file=test_files[0], filetype='eq')

CPU times: user 5 µs, sys: 9 µs, total: 14 µs
Wall time: 18.1 µs


In [9]:
# Let's have a look at the output file. 
!cat holman_ic

tstart 2456117.641933589
tstep +20.0
trange 600.
geocentric 0
state
-2.093834952466474e+00  1.000913720009255e+00  4.197984954533551e-01 
-4.226738336365523e-03 -9.129140909705197e-03 -3.627121453928710e-03 


# Lower-level functionalities
 - The above "ParseElements" instantiation uses some underlying functionality to perform the parsing & storage of the elements.
 - Here we demonstrate some of the underlying functionality.

##### parse_input.ParseElements()
 - With no arguments to set up an empty object. 

In [10]:
P = parse_input.ParseElements()
# List the methods contained within ParseElements
[i for i in dir(P) if '__' not in i]

Keywords 'input_file' and/or 'filetype' missing; initiating empty object.


['make_bary_equatorial', 'parse_ele220', 'parse_orbfit', 'save_elements']

##### parse_input.ParseElements.parse_orbfit(felfile)
 - Parses a single orbit file (.fel or .eq) and stores time & the heliocentric ecliptic coordinates. 

In [11]:
P.parse_orbfit(felfile=test_files[1])
# P now has some non-method attributes!
print([i for i in dir(P) if '__' not in i])
# See the content:
P.__dict__

['heliocentric_ecliptic_cartesian_elements', 'make_bary_equatorial', 'parse_ele220', 'parse_orbfit', 'save_elements', 'time']


{'time': <Time object: scale='tt' format='mjd' value=56184.25284321>,
 'heliocentric_ecliptic_cartesian_elements': {'x_HelioEcl': -3.141624354767198,
  'dx_HelioEcl': -0.00561642987380069,
  'y_HelioEcl': 3.883199812201892,
  'dy_HelioEcl': -0.004532863527910482,
  'z_HelioEcl': 2.191304909819448,
  'dz_HelioEcl': 0.0001807300513403061,
  'sigma_x_HelioEcl': '5.371149655803086E-13',
  'sigma_dx_HelioEcl': '4.850526570879810E-19',
  'sigma_y_HelioEcl': '3.083965957221651E-13',
  'sigma_dy_HelioEcl': '3.215811356890292E-19',
  'sigma_z_HelioEcl': '4.296172191351023E-13',
  'sigma_dz_HelioEcl': '6.147316166044891E-19',
  'x_y_HelioEcl': '2.354557490078648E-13',
  'x_z_HelioEcl': '6.163733473770746E-14',
  'x_dx_HelioEcl': '-4.310096570860611E-16',
  'x_dy_HelioEcl': '2.646251750285150E-16',
  'x_dz_HelioEcl': '1.741935780011843E-16',
  'y_z_HelioEcl': '-1.308201104779745E-13',
  'y_dx_HelioEcl': '-1.500616034123045E-16',
  'y_dy_HelioEcl': '2.338140588946915E-16',
  'y_dz_HelioEcl': '1.40

##### parse_input.ParseElements.make_bary_equatorial()
 - Convert whatever is available (currently only Heliocentric Ecliptic cartesian implemented) to Barycentric Equatorial cartesian elements. 

In [12]:
P.make_bary_equatorial()
# P now has some more attributes! See the content:
P.barycentric_equatorial_cartesian_elements

{'x_BaryEqu': -3.1435635433696,
 'y_BaryEqu': 2.6890636461132758,
 'z_BaryEqu': 3.5542111848815767,
 'dx_BaryEqu': -0.005610620819862402,
 'dy_BaryEqu': -0.004232958051824348,
 'dz_BaryEqu': -0.0016383640293136613}

##### parse_input.ParseElements.save_elements(output_file)
 - Save the barycentric equatorial cartesian elements to file. 
 - Default filename is holman_ic, but this can be changed with output_file keyword.

In [13]:
P.save_elements(output_file='see_this_works')
# Let's have a look at the output file. 
!cat see_this_works

tstart 2456184.7528431923
tstep +20.0
trange 600.
geocentric 0
state
-3.143563543369600e+00  2.689063646113276e+00  3.554211184881577e+00 
-5.610620819862402e-03 -4.232958051824348e-03 -1.638364029313661e-03 


##### parse_input.equatorial_helio2bary(input_xyz)
 - Used by ParseElements.make_bary_equatorial()
 - Convert a cartesian vector from mean equatorial heliocentric to barycentric.

In [14]:
# Set up input using astroquery.jplhorizons.Horizons:
# Geocenter at 2020-Mar-28 12:00:00 TDB, equatorial:
hor_in_table = Horizons('2011 QF99', '500@10', 2458937.0, id_type='smallbody'
                        ).vectors(refplane='earth')['x', 'y', 'z', 'vx', 'vy', 'vz']
input_xyz = list(hor_in_table.as_array()[0])
# Set up expected output, ie. Horizons answer:
hor_out_table = Horizons('2011 QF99', '500@0', 2458937.0, id_type='smallbody'
                         ).vectors(refplane='earth')['x', 'y', 'z', 'vx', 'vy', 'vz']
expected_output_xyz = np.array(list(hor_out_table.as_array()[0]))
# Get actual output from parsing and converting input:
output_xyz = parse_input.equatorial_helio2bary(input_xyz, 2458937.0)
# Check the discrepancy:
error = np.abs(expected_output_xyz - output_xyz)
print('Position error: ', error[:3] * au_km * 1e6, 'mm')
print('Velocity error: ', error[3:6] * au_km * 1e6, 'mm/day')

Position error:  [0.1328696 0.2657392 0.       ] mm
Velocity error:  [0.00000000e+00 6.48777346e-05 4.05485841e-06] mm/day


##### parse_input.ecliptic_to_equatorial(input_xyz)
 - Used by ParseElements.make_bary_equatorial()
 - Convert a cartesian vector from mean ecliptic to mean equatorial.

In [15]:
# Set up input using astroquery.jplhorizons.Horizons:
# Geocenter at 2020-Mar-28 12:00:00 TDB, equatorial:
hor_table = Horizons('2011 QF99', '500@10', 2458937.0, id_type='smallbody')
hor_in_table = hor_table.vectors(refplane='ecliptic')['x', 'y', 'z', 'vx', 'vy', 'vz']
input_xyz = list(hor_in_table.as_array()[0])
# Set up expected output, ie. Horizons answer:
hor_out_table = hor_table.vectors(refplane='earth')['x', 'y', 'z', 'vx', 'vy', 'vz']
expected_output_xyz = np.array(list(hor_out_table.as_array()[0]))
# Get actual output from parsing and converting input:
output_xyz = parse_input.ecliptic_to_equatorial(input_xyz)
# Check the discrepancy:
error = np.abs(expected_output_xyz - output_xyz)
print('Position error: ', error[:3] * au_km * 1e6, 'mm')
print('Velocity error: ', error[3:6] * au_km * 1e6, 'mm/day')

Position error:  [0.        0.2657392 0.2657392] mm
Velocity error:  [0.00000000e+00 6.48777346e-05 1.21645752e-05] mm/day
